In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import re
import csv
import time
from google.colab import drive
import matplotlib.pyplot as plt
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
statsList = ['stats','keepers', 'keepersadv', 'shooting', 'passing', 'passing_types', 'gca', 'defense', 'possession', 'playingtime', 'misc']

compid = ['9','12','20','11','13']

endpartdict = {'9':'Premier-League-Stats','12':'-La-Liga-Stats','20':'-Bundesliga-Stats','11':'-Serie-A-Stats','13':'-Ligue-1-Stats'}

statdict = { "stats":"standard","keepers":"keeper","keepersadv":"keeper_adv","shooting" : "shooting", "passing": "passing", "passing_types" : "passing_types", "gca":"gca" , "defense":"defense", "possession": "possession","playingtime":"playing_time" , "misc":"misc"}

location = "/content/drive/MyDrive/dataasetcurrent/"


In [ ]:
#function which gets the table data scraped by passing the url link
def getplayerstats(urlstat:str,compid:int, scrapestat:str):
    print("->",urlstat,compid,scrapestat)
    url = f"https://fbref.com/en/comps/{compid}/{urlstat}/"
    print(f"Data scraped from {url}")
    time.sleep(5)
    res = requests.get(url,timeout=10)
    comm = re.compile("<!--|-->")
    soup = bs(comm.sub("", res.text), 'html.parser', from_encoding='utf-8')
    table = soup.find("div", {"id": f"div_stats_{scrapestat}"})
    df = pd.read_html(str(table))
    df = df[0]
    return df


In [ ]:
#template function to get the data by passing in url and storing it as excel file

def getplayerstats(urlstat:str,compid:int, scrapestat:str ):
    if urlstat not in statsList:
        raise ValueError(f"stat must be one of {statsList}")
    table = getscrapetable(urlstat, compid, scrapestat)
    #print(table)
    return _getDataframe(table)

In [ ]:

def getscrapetable(urlstat:str ,compid:int, scrapestat:str):
    print("->",urlstat,compid,scrapestat)
    url = f"https://fbref.com/en/comps/{compid}/{urlstat}/"
    print(f"Getting data from {url}")
    time.sleep(5)
    res = requests.get(url,timeout=10)
    comm = re.compile("<!--|-->")
    soup = bs(comm.sub("", res.text), 'html.parser', from_encoding='utf-8')
    #soup = BeautifulSoup(html, )
    return soup.find("div", {"id": f"div_stats_{scrapestat}"})


def _getDataframe(table):
    df = pd.read_html(str(table))
    df = df[0]
    return df

In [ ]:
#get current year stats
comp = '19'
for urlstatdet in statdict.keys():
        print (comp + " : " + urlstatdet)
        # get value of the corresponding urlstat from dictionary to get the scrape stat detail
        scrapestatdet = statdict[urlstatdet]
        table = getplayerstats(urlstatdet,comp,scrapestatdet)
        file = urlstatdet + comp+'.csv'
        filename = location + file
        #filename = 'dataal.csv'
        print(filename)
        table.to_csv(filename, index=False)
        filename = ""
        print("----------nexxt----")
        #print(table)
        table.drop(table.index, inplace=True)

In [ ]:
#get stats by specifying the year
year1 = 2016
year2 = 2017
finaldf = pd.DataFrame({})

for i in range(0,5):
        val1 = str(year1 + i)
        val2 = str(year2 + i)
        print("years",val1,val2)
        for comp in compid:
            for urlstatdet in statdict.keys():

                scrapestatdet = statdict[urlstatdet]
                time.sleep(10)
                value = endpartdict[comp]
                url1 = f"https://fbref.com/en/comps/{comp}/{val1}-{val2}/{urlstatdet}/{val1}-{val2}{value}"
                print(url1)
                res = requests.get(url1,timeout=10)
                comm = re.compile("<!--|-->")
                soup = bs(comm.sub("", res.text), 'lxml')
                print(f"div_stats_{scrapestatdet}")
                tableprev = soup.find("div", {"id": f"div_stats_{scrapestatdet}"})
                tempdf = pd.read_html(str(tableprev))
                df = tempdf[0]
                #print(df)
                tableprev.decompose()
                fileprev = urlstatdet + comp + val1+ '-' +val2 +'.csv'
                filenameprev = location + fileprev
                df.to_csv(filenameprev, index=False)
                df.drop(df.index, inplace=True)
                print(filenameprev)

years 2016 2017
https://fbref.com/en/comps/9/2016-2017/stats/2016-2017Premier-League-Stats
div_stats_standard
/content/drive/MyDrive/dataasetcurrent/latest/stats92016-2017.csv
https://fbref.com/en/comps/9/2016-2017/keepers/2016-2017Premier-League-Stats
div_stats_keeper
/content/drive/MyDrive/dataasetcurrent/latest/keepers92016-2017.csv


KeyboardInterrupt: ignored